In [2]:
!pip install nltk
!pip install gensim

In [3]:
cd drive/My Drive/CSCI566_project/memes900k

/content/drive/My Drive/CSCI566_project/memes900k


In [4]:
selected = ['african children dancing','Angry Cat Meme',"Anchorman Birthday","Annoying Gamer Kid","Awkward Seal","Black Kid","burning house girl","Desk Flip Rage Guy","evil plan kid","FU*CK THAT GUY","Grumpy Cat 2","Grumpy Cat Santa Hat","high/drunk guy","Joseph Ducreux","Skeptical african kid","so doge","Success Kid","Willy Wonka","You shall not pass","Y U No"]

In [141]:
x_train = []
y_train = []
# select from github dataset
with open("captions.txt") as file:
  for line in file.readlines():
    l,_,text = line.split('\t')
    y_train.append(l.strip())
    text = text.strip().replace('<sep>',"[SEP]")
    x_train.append(text)
# with open("captions_train.txt") as file:
#   for line in file.readlines():
#     l,_,text = line.split('\t')
#     if l.strip() in selected:
#       y_train.append(l.strip())
#       text = text.strip().replace('<sep>',"")
#       x_train.append(text)

In [142]:
templates_map = dict(zip(set(y_train),range(300)))
templates_map_reverse = {v:k for k,v in templates_map.items()}
len(templates_map)

300

In [143]:
y_train = [templates_map[i] for i in y_train if i ]

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer

In [ ]:
porter_stemmer = PorterStemmer()
def clean_text(text):
  new = simple_preprocess(remove_stopwords(text),max_len=50)
  new = [porter_stemmer.stem(word) for word in new]
  return new


In [32]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
tokenizer = Tokenizer()


In [144]:
x_input = list(map(clean_text,x_train))

In [145]:
tokenizer.fit_on_texts(x_input)
x_token = tokenizer.texts_to_sequences(x_input)
X_train_NN = pad_sequences(x_token, maxlen=50)

In [146]:
X_train_NN[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 3638,    1,   53,
          32,   18,  345, 3287,  111,  309],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 4785,    1,   53,
          32,   18,   77,  209,    9, 1110]], dtype=int32)

In [154]:
num_index = len(tokenizer.word_index)
EMBEDDING_DIM = 300

In [41]:
# from keras.models import Sequential
# from keras.layers import Embedding

# model = Sequential()
# model.add(Embedding(
#   input_dim=num_index+1, 
#   output_dim=20, 
#   input_length=50))

# model.compile('rmsprop', 'mse')
# output_array = model.predict(X_train_NN)[0]

In [58]:
x_output=model.predict(X_train_NN)


In [147]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
y_train_ohe=onehotencoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()

In [59]:
len(x_output)

50000

In [148]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(X_train_NN,
                                                   y_train_ohe,
                                                   test_size = 0.1,
                                                   random_state = 1,stratify=y_train)

In [ ]:
from collections import Counter
print(Counter(train_y))
print(Counter(test_y))

In [150]:
# Deep learning: 
from keras.models import Input, Model
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate
from keras.layers import Bidirectional

class RnnModel():
    def __init__(self, num_index, output_dim, max_len,num_labels, X_additional=None):
        
        inp1 = Input(shape=(max_len,))
        x = Embedding(num_index+1, output_dim=output_dim, input_length=max_len)(inp1)
        x = LSTM(256)(x)
        # x = Bidirectional(LSTM(256, return_sequences=True))(x)
        # x = Bidirectional(LSTM(150))(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(64, activation="relu")(x)
        x = Dense(num_labels, activation="sigmoid")(x)    
        model = Model(inputs=inp1, outputs=x)

        model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['acc'])
        self.model = model

In [109]:
len(data_whole_ohe[0])

20

In [155]:
rnn = RnnModel(num_index,300,50,300)

In [ ]:
rnn.model.fit(
            train_x,
            train_y, 
            batch_size=16, 
            epochs=4,
            validation_data=(test_x, test_y),
        )

Epoch 1/4
 6933/50625 [===>..........................] - ETA: 11:33:37 - loss: 4.2781 - acc: 0.1751

In [139]:
(y_train_ohe.argmax(axis=1) == np.array(y_train)).sum()

50000

In [118]:
rnn.model.evaluate(test_x,test_y)

157/157 [==============================] - 13s 82ms/step - loss: 1.5388 - acc: 0.5528


[1.5388253927230835, 0.5527999997138977]

In [132]:
rnn.model.predict(test_x[:2])

array([[5.5675805e-03, 2.0642579e-03, 8.1730396e-02, 8.3740675e-01,
        2.8199852e-03, 1.1036038e-02, 2.5509000e-03, 3.1282604e-03,
        8.4766746e-04, 4.1088462e-03, 7.5268149e-03, 5.9533417e-03,
        9.6043348e-03, 2.3640901e-02, 2.2493601e-03, 5.8844507e-02,
        5.4475665e-04, 5.7902932e-03, 5.7570696e-02, 6.1917305e-04],
       [3.5451710e-02, 1.6262293e-02, 5.8132410e-04, 1.0389864e-02,
        2.6842177e-02, 2.3312151e-02, 2.4857879e-02, 5.2195489e-03,
        6.4093471e-03, 4.7499835e-02, 1.0558933e-02, 3.1789273e-02,
        2.7622938e-02, 2.7856678e-02, 2.5273263e-03, 2.7631462e-02,
        6.2694025e-01, 2.4215281e-02, 1.6701937e-02, 1.5996903e-02]],
      dtype=float32)

In [133]:
np.argmax([5.5675805e-03, 2.0642579e-03, 8.1730396e-02, 8.3740675e-01,
        2.8199852e-03, 1.1036038e-02, 2.5509000e-03, 3.1282604e-03,
        8.4766746e-04, 4.1088462e-03, 7.5268149e-03, 5.9533417e-03,
        9.6043348e-03, 2.3640901e-02, 2.2493601e-03, 5.8844507e-02,
        5.4475665e-04, 5.7902932e-03, 5.7570696e-02, 6.1917305e-04])

3